In [10]:
import pyttsx3
import datetime
import wikipedia
import subprocess
import webbrowser
import pywhatkit as wk
import os
import cv2
import pyautogui
import time
import speech_recognition as sr
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib
import pyaudio

In [11]:
engine = pyttsx3.init('sapi5')
voices = engine.getProperty('voices')
engine.setProperty('voice',voices[1].id)
engine.setProperty('rate',150)

In [12]:
def speak(audio):
    engine.say(audio)
    engine.runAndWait()

def takeCommand():

    r = sr.Recognizer()

    while True:
        with sr.Microphone() as source:
            print("Listening")
            #r.pause_threshold = 1
            r.adjust_for_ambient_noise(source, duration=1)
            audio = r.listen(source)

        try:
            print("Recognizing")
            query = r.recognize_google(audio,language='en-in')
            print(f"User said: {query}\n")
            return query.lower()

        except Exception as e:
            #print("please say it again", e)
            #speak("please say it again")
            return "None"

In [13]:
def username():
    speak("What should I call you?")
    uname = takeCommand()
    if uname.lower() != 'none':
        speak("Welcome " + uname)
        speak("How can i assist you today.")
    else:
        speak("I'm sorry, I didn't catch your name.")
        username()

def wishMe():
    hour = datetime.datetime.now().hour
    if 0 <= hour < 12:
        speak("Good Morning")
    elif 12 <= hour < 18:
        speak("Good Afternoon")
    else:
        speak("Good Evening") 
    speak("I am your virtual assistant violet")


In [14]:
def open_in_browser(url, browser='chrome'):
    try:
        if browser.lower() == 'brave':
            subprocess.Popen([r'C:\Program Files (x86)\Internet Explorer\iexplore.exe', url])
        else:
            webbrowser.open(url)
    except Exception as e:
        print(f"Error opening in {browser}: {e}")

In [15]:
app_paths = {
    "firefox": r"C:\Program Files\Mozilla Firefox\firefox.exe",
    "chrome": r"C:\Program Files\Google\Chrome\Application\chrome.exe",
    "calculator": r'C:\Windows\System32\calc.exe',
    "notepad": r'C:\Windows\System32\notepad.exe',
    "file explorer": r'C:\Windows\explorer',
    "desktop folder": r'C:\Users\abhin\OneDrive\Desktop',
    "downloads folder": r'C:\Users\abhin\Downloads',
    "documents folder": r'C:\Users\abhin\OneDrive\Documents',
    "microsoft edge": r'C:\Program Files (x86)\Internet Explorer\iexplore.exe',
    "control panel": r'C:\Windows\System32\control.exe',
    "word": r'C:\ProgramData\Microsoft\Windows\Start Menu\Programs\Word 2016.lnk',
    "excel": r'C:\ProgramData\Microsoft\Windows\Start Menu\Programs\Excel 2016.lnk',
    "powerpoint": r'C:\ProgramData\Microsoft\Windows\Start Menu\Programs\PowerPoint 2016.lnk',
    "outlook": r'C:\ProgramData\Microsoft\Windows\Start Menu\Programs\Outlook 2016.lnk',
    "sublime text": r'C:\ProgramData\Microsoft\Windows\Start Menu\Programs\Sublime Text.lnk',
    "vs code": r"C:\Users\abhin\AppData\Local\Programs\Microsoft VS Code\Code.exe",
    "anaconda": r"C:\Users\abhin\AppData\Roaming\Microsoft\Windows\Start Menu\Programs\Anaconda3 (64-bit)\Anaconda Navigator (anaconda3).lnk",
    "jupyter notebook": r"C:\Users\abhin\AppData\Roaming\Microsoft\Windows\Start Menu\Programs\Anaconda3 (64-bit)\Jupyter Notebook (anaconda3).lnk"
    
    
    # Add more app paths as needed
}

In [16]:
def open_app(app_name):
    if app_name.lower() in app_paths:
        subprocess.Popen(['explorer', app_paths[app_name.lower()]])
    else:
        print("Sorry, I couldn't find that app.")

In [17]:
def send_email(subject, message, to_email):
    # Set up the SMTP server
    smtp_server = "smtp.gmail.com"
    smtp_port = 587  # For SSL, use 465

    # Email and password for the sender's Gmail account
    email = "abhinandan1903@gmail.com"
    password = "kucz eqhm jjlz qtim"

    # Create a multipart message and set headers
    msg = MIMEMultipart()
    msg['From'] = email
    msg['To'] = to_email
    msg['Subject'] = subject

    # Add message body
    msg.attach(MIMEText(message, 'plain'))

    # Start TLS encryption and login to the SMTP server
    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()
    server.login(email, password)

    # Send email
    text = msg.as_string()
    server.sendmail(email, to_email, text)

    # Quit server
    server.quit()
    
def send_email_command():
    speak("Please enter the recipient's email address.")
    recipient_email = input("Recipient's Email: ")
    if "@" not in recipient_email:
        speak("Invalid email format. Please enter a valid email address.")
        return
    speak("What should be the subject of the email?")
    subject = takeCommand()
    speak("What message do you want to send?")
    message = takeCommand()
    
    # Ask if the user wants to add anything else to the message
    speak("Do you want to say anything else?")
    additional_message = takeCommand()
    if additional_message and additional_message != "no":
        message += " " + additional_message
    
    # Confirm before sending the email
    speak("Do you want to send this email?")
    confirmation = takeCommand()
    if confirmation and confirmation != "no":
        send_email(subject, message, recipient_email)
        speak("Email sent successfully!")
    else:
        speak("Email not sent.")

In [22]:
if __name__ == "__main__":
    wishMe()
    username()
    
    while True:
        query = takeCommand().lower()  
        
        if 'violet' in query:
            response="yes"
            print(response)
            speak(response)

        elif 'just open google' in query:
            open_in_browser('https://www.google.com', browser='brave')

        elif 'just open youtube' in query:
            open_in_browser('https://www.youtube.com', browser='brave')
           
        elif 'search on google' in query:
            speak("What should I search?")
            search_query = takeCommand().lower()
            search_url = f"https://www.google.com/search?q={search_query.replace(' ', '+')}"
    
            webbrowser.open(search_url)
    
            try:
                results = wikipedia.summary(search_query, sentences=2)
                speak(results)
            except wikipedia.exceptions.DisambiguationError as e:
                speak("I found multiple results. Please be more specific.")
            except wikipedia.exceptions.PageError as e:
                speak("I'm sorry, but I couldn't find any information on that topic.")

        elif 'open youtube' in query:
            speak("What would you like to watch?")
            video_query = takeCommand().lower()
            wk.playonyt(f"{video_query}")

        elif 'search on youtube' in query:
            query=query.replace("search on youtube","")
            webbrowser.open(f"www.youtube.com/results?search_query={query}")


        elif 'who are you' in query:
            response='my name is violet'
            print(response)
            speak(response)
            print('I can do everything that my creator programmed me to do')
            speak('I can do everything that my creator programmed me to do')

        elif 'who created you' in query:
            response='I dont know her name I am created with python language in visual studio code'
            print(response)
            speak(response)

        elif 'how are you' in query:
            response = "I don't have feelings, but I'm here and ready to assist you. How can I help?"
            print(response)
            speak(response)

        elif 'what can you do' in query:
            response = "I can perform tasks for you"
            print(response)
            speak(response)
        
        elif 'what are my system specifications' in query:
            response="device name LAPTOP-EU6FGMJK , RAM 8.00 GB (7.85 GB usable)  system type 64-bit operating system, x64-based processor , No pen or touch input is available for this display "
            print(response)
            speak(response)
        
        elif 'ok' in query: 
            response="anything else I can help you with"
            print(response)
            speak(response)

        elif 'thank you' in query:
            response = "You're welcome! If you have any more questions or need assistance, feel free to ask."
            print(response)
            speak(response)

        elif 'bye' in query:
            response = "bye ! have a good day"
            print(response)
            speak(response)

        elif 'what is' in query:
            try:
                speak("Searching Wikipedia...")
                query = query.replace("what is", "").strip()
                results = wikipedia.summary(query, sentences=2)
                speak("According to Wikipedia")
                print(results)
                speak(results)
            except wikipedia.exceptions.DisambiguationError as e:
                speak("I found multiple results. Please be more specific.")
            except wikipedia.exceptions.PageError as e:
                speak("I'm sorry, but I couldn't find any information on that topic.")

        elif 'who is' in query:
            try:
                speak("Searching Wikipedia...")
                query = query.replace("who is", "").strip()
                results = wikipedia.summary(query, sentences=2)
                speak("According to Wikipedia")
                print(results)
                speak(results)
            except wikipedia.exceptions.DisambiguationError as e:
                speak("I found multiple results. Please be more specific.")
            except wikipedia.exceptions.PageError as e:
                speak("I'm sorry, but I couldn't find any information on that topic.")

        elif 'where is' in query:
            try:
                speak("Searching Wikipedia...")
                query = query.replace("where is", "").strip()
                results = wikipedia.summary(query, sentences=2)
                speak("According to Wikipedia")
                print(results)
                speak(results)
            except wikipedia.exceptions.DisambiguationError as e:
                speak("I found multiple results. Please be more specific.")
            except wikipedia.exceptions.PageError as e:
                speak("I'm sorry, but I couldn't find any information on that topic.")
              
        elif 'open camera' in query:
            speak("opening")
            print("opening...")
            cap=cv2.VideoCapture(0)
            while True:
                ret, img = cap.read()
                cv2.imshow('webcam',img)
                k = cv2.waitKey(1)
                if k == 27:
                    break
            cap.release()
            cv2.destroyAllWindows()
            
        elif 'take screenshot' in query:            
            speak('Tell me a name for the file')
            name = takeCommand().lower()
            save_directory = r'C:\Users\abhin\OneDrive\Pictures\Screenshots'  
            if not os.path.exists(save_directory):
                os.makedirs(save_directory)  
            time.sleep(3) 
            img = pyautogui.screenshot()            
            filename = f"{name}.png"
            file_path = os.path.join(save_directory, filename)
            img.save(file_path)
            speak("Screenshot saved as {}".format(filename))
            

        elif 'close browser' in query:
            os.system("taskkill/f /im brave.exe")

        elif 'open command prompt' in query:
            os.system("start cmd")
            speak("opening...")
            print("opening...")

        elif 'close command prompt' in query:
            os.system("taskkill/f /im cmd.exe")
        
        elif 'exact time now' in query:
            strTime = datetime.datetime.now().strftime("%H:%M:%S")
            speak(f"the time is {strTime}")

        elif 'open' in query:
            app_name = query.split("open", 1)[1].strip()  # Extract the app name from the query
            if app_name.lower() in app_paths:
                open_app(app_name)
                print("opening...")
                speak("opening...")
            else:
                print("Sorry, I couldn't find that app.")
                speak("Sorry, I couldn't find that app.")

        elif 'shutdown the system' in query:
            os.system("shutdown /s /t 5")

        elif 'restart the system' in query:
            os.system("shutdown /r /t 5")
        
        elif 'lock the system' in query:
            os.system("rundll32.exe user32.dll,LockWorkStation")
            
        elif 'mute' in query or 'unmute' in query:
            pyautogui.press("volumemute")
            
        elif 'volume up' in query:
            pyautogui.press("volumeup")
            pyautogui.press("volumeup")
            pyautogui.press("volumeup")
            pyautogui.press("volumeup")
            pyautogui.press("volumeup")
        
        elif 'volume down' in query:
            pyautogui.press("volumedown")
            pyautogui.press("volumedown")
            pyautogui.press("volumedown")
            pyautogui.press("volumedown")
            pyautogui.press("volumedown")
        
        elif 'send email' in query:
            send_email_command()
                
        elif 'exit' in query:
            print("Goodbye ! have a nice day.")
            speak("Goodbye ! have a nice day.")
            break                  
                         
        else:
            speak("Sorry, I didn't get you. Could you please repeat?")
            print("Sorry, I didn't get you. Could you please repeat?")


Listening
Recognizing
User said: Abhinandan

Listening
Recognizing
User said: open Firefox

opening...
Listening
Recognizing
Sorry, I didn't get you. Could you please repeat?
Listening
Recognizing
User said: open Chrome

opening...
Listening
Recognizing
User said: open calculator

opening...
Listening
Recognizing
User said: open Notepad

opening...
Listening
Recognizing
User said: open file explorer

opening...
Listening
Recognizing
User said: open desktop folder

opening...
Listening
Recognizing
User said: open downloads folder

opening...
Listening
Recognizing
User said: open documents folder

opening...
Listening
Recognizing
User said: open Microsoft edge

opening...
Listening
Recognizing
User said: open control panel

opening...
Listening
Recognizing
User said: open

Sorry, I couldn't find that app.
Listening
Recognizing
User said: open Excel

opening...
Listening
Recognizing
User said: open PowerPoint

opening...
Listening
Recognizing
User said: open Outlook

anything else I can h